<a href="https://colab.research.google.com/github/pfauda/python/blob/gh-pages/EstrategiaRSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Estrategia RSI con activos de alta volatilidad

In [2]:
!pip install yfinance
!pip install mplfinance

     |████████████████████████████████| 5.5MB 11.9MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=423ad1a364660f2edd4ae2cb1ca28f246ff337f6c71e07d1d5bae79ac2549510
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 61kB 5.4MB/s 


In [3]:
# Importar librerias
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import copy
import matplotlib.pyplot as plt
from matplotlib.dates import date2num
from mplfinance.original_flavor import candlestick_ohlc
import datetime

In [4]:
# Inicializaciones
tickers = ["GGAL", "BMA", "SUPV", "BBAR"]
#tickers = ["GGAL"]
precios = {} # diccionario para guardar los datos

# Importación de datos
for i in range(len(tickers)):
    precios[tickers[i]] = yf.download(tickers[i], start="2000-01-01") #["Adj Close"].to_frame()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
def calc_rsi(ohlc: pd.DataFrame, period: int = 14, source: str = 'Close') -> pd.Series:
    """See source https://github.com/peerchemist/finta
    and fix https://www.tradingview.com/wiki/Talk:Relative_Strength_Index_(RSI)
    Relative Strength Index (RSI) is a momentum oscillator that measures the speed and change of price movements.
    RSI oscillates between zero and 100. Traditionally, and according to Wilder, RSI is considered overbought when above 70 and oversold when below 30.
    Signals can also be generated by looking for divergences, failure swings and centerline crossovers.
    RSI can also be used to identify the general trend."""

    delta = ohlc[source].diff()

    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    gain = up.ewm(com=(period - 1), min_periods=period).mean()
    loss = down.abs().ewm(com=(period - 1), min_periods=period).mean()

    RS = gain / loss
    return pd.Series(100 - (100 / (1 + RS)), name="RSI")

In [6]:
# Cálculo RSI
price_dict = copy.deepcopy(precios)
for ticker in tickers:
    print("Calculando RSI de", ticker)
    price_dict[ticker]["RSI"] = calc_rsi(price_dict[ticker], 14, 'Close')
    price_dict[ticker].dropna(inplace=True)

Calculando RSI de GGAL
Calculando RSI de BMA
Calculando RSI de SUPV
Calculando RSI de BBAR


In [7]:
def plot_chart(data, n, ticker, sobreVenta, sobreCompra):
    
    # Filter number of observations to plot
    data = data.iloc[-n:]
    
    # Create figure and set axes for subplots
    fig = plt.figure()
    fig.set_size_inches((18, 7))
    #ax_candle = fig.add_axes((0, 0.72, 1, 0.32))
    ##ax_macd  = fig.add_axes((0, 0.48, 1, 0.2), sharex=ax_candle)
    #ax_rsi    = fig.add_axes((0, 0.24, 1, 0.2), sharex=ax_candle)
    #ax_vol    = fig.add_axes((0, 0, 1, 0.2), sharex=ax_candle)
    ax_candle = fig.add_axes((0, 1.00, 1, 0.50))
    ax_rsi    = fig.add_axes((0, 0.75, 1, 0.20), sharex=ax_candle)
    ax_vol    = fig.add_axes((0, 0.50, 1, 0.20), sharex=ax_candle)
    
    # Format x-axis ticks as dates
    ax_candle.xaxis_date()
    
    # Get nested list of date, open, high, low and close prices
    ohlc = []
    for date, row in data.iterrows():
        openp, highp, lowp, closep = row[:4]
        ohlc.append([date2num(date), openp, highp, lowp, closep])
 
    ## Plot candlestick chart
    #ax_candle.plot(data.index, data["ma10"], label="MA10")
    #ax_candle.plot(data.index, data["ma30"], label="MA30")
    candlestick_ohlc(ax_candle, ohlc, colorup="g", colordown="r", width=0.8)
    #ax_candle.legend()
    
    ## Plot MACD
    #ax_macd.plot(data.index, data["macd"], label="macd")
    #ax_macd.bar(data.index, data["macd_hist"] * 3, label="hist")
    #ax_macd.plot(data.index, data["macd_signal"], label="signal")
    #ax_macd.legend()
    
    # Plot RSI
    # Above 70% = overbought, below 30% = oversold
    ax_rsi.set_ylabel("(%)")
    ax_rsi.plot(data.index, [sobreCompra] * len(data.index), label="S.Compra")
    ax_rsi.plot(data.index, [sobreVenta] * len(data.index), label="S.Venta")
    ax_rsi.plot(data.index, data["RSI"], label="RSI")
    ax_rsi.legend()
    
    # Show volume in millions
    ax_vol.bar(data.index, data["Volume"] / 1000000)
    ax_vol.set_ylabel("(En millones)")
   
    # Save the chart as PNG
    #fig.savefig("charts/" + ticker + ".png", bbox_inches="tight")
    
    plt.show()

In [8]:
#plot_chart(price_dict[ticker], 7000, ticker, 20, 40)

In [9]:
class Strategy:

  def __init__(self, data, capital, source):
    self.data = data
    self.capital = capital
    self.source = source
    self.date_buy = data.index[0]
    self.date_close = data.index[0]
    self.days = 0
    self.profit_abs = 0
    self.profit_prc = 0
    self.cum_profit_abs = 0
    self.cum_profit_prc = 0

    self.trades = pd.DataFrame(columns = ['date_buy', 'date_close', 'days', 
                                          'price_buy', 'price_close', 'shares',
                                          'profit_abs', 'profit_prc',
                                          'cum_profit_abs', 'cum_profit_prc'])
    self.entry_id = ""
    #self.balance = []
    #self.summary = pd.DataFrame(columns = ["ret", "ret_cum", "max", "drowdown"])
    self.len = len(data) - 1
    self.entry_candle = 0

  #def compute_tick(self, i):
  #  if self.entry_id == "":
  #    self.balance.append(0)
  #  else:
  #    self.balance.append((self.data['Close'][self.entry_candle]/self.data[self.source][i-1])-1)

  def entry(self, entry_id, i):
    if self.entry_id != entry_id:
      if i < self.len:
        self.entry_candle = i
        self.entry_id = entry_id
        self.date_buy = self.data.index[i+1]
        # Para comprar se toma el precio de cierre del dia anterior
        self.shares_old = int( ( self.capital + self.cum_profit_abs ) / self.data['Close'][i] )

  def close_all(self, i):
    if self.entry_id != "":
      if i < self.len:
        self.entry_id = ""
        self.date_close = self.data.index[i+1]
        self.days = (self.date_close - self.date_buy).days
        # Para el calculo de la ganancia tomar el precio de apertura de la estrategia del dia siguiente
        self.profit_abs = self.shares_old * ( self.data['Open'][i+1] - self.data['Open'][self.entry_candle+1] )
        self.profit_prc = (self.data['Open'][i+1] / self.data['Open'][self.entry_candle+1] - 1)
        self.cum_profit_abs += self.profit_abs
        self.cum_profit_prc += self.profit_prc
        self.trades = self.trades.append({'date_buy': self.date_buy,
                                          'date_close': self.date_close,
                                          'days': self.days,
                                          'price_buy': self.data['Open'][self.entry_candle+1],
                                          'price_close': self.data['Open'][i+1],
                                          'shares': self.shares_old,
                                          'profit_abs': self.profit_abs,
                                          'profit_prc': self.profit_prc,
                                          'cum_profit_abs': self.cum_profit_abs,
                                          'cum_profit_prc': self.cum_profit_prc},
                                          ignore_index=True)
        self.shares_old = 0
        self.entry_candle = 0

  def end(self, i):
    self.close_all(i)

  def print_report(self):
    if len(self.trades) > 0:
      pd.set_option("display.max_rows", None, "display.max_columns", 200)
      print('\nTrades realizados')
      print('-------------------------------------------------------------------------')
      print(self.trades)
      print('-------------------------------------------------------------------------')
      print("\nGanancia acumulada")
      print(round(self.cum_profit_abs, 2))

      print(f'\nLa ganancia promedio es de {round(self.trades["profit_prc"].mean()*100, 2)}%')
      print(f'La desviación estandar del resultado es de {round(self.trades["profit_prc"].std()*100, 2)}%')
      print(f'La duración promedio es de {round(self.trades["days"].mean(), 2)} días')
      print(f'El {round((len(self.trades[self.trades["profit_prc"]>0])/len(self.trades))*100, 2)}% de las veces se sale con ganancias')
      print(f'El {round((len(self.trades[self.trades["profit_prc"]<=0])/len(self.trades))*100, 2)}% de las veces se sale con perdidas')
      print(f'El peor resultado al cierre fue de {round(self.trades["profit_prc"].min()*100, 2)}%')
      print(f'El mejor resultado al cierre fue de {round(self.trades["profit_prc"].max()*100, 2)}%')

      #self.summary["ret"] = np.array(self.balance)
      #self.summary["ret_cum"] = (1+self.summary["ret"]).cumprod()

      #print("\nbajas durante la estrategia")
      #self.summary["max"] = self.summary["ret_cum"].cummax()
      #self.summary["drowdown"] = self.summary["ret_cum"] / self.summary["max"] -1
      #print(f'baja máxima de {round(self.summary["drowdown"].min()*100, 2)}%')

    # RSI Actuales
    print("\nRSI actuales")
    print(round(self.data["RSI"][-1], 2))

  def get_result(self) -> float:
    return self.cum_profit_abs

In [10]:
def calc_strategy_rsi(data, sobreVenta, sobreCompra, source) -> float:

  strategy = Strategy(data, 100000, source)

  for i in range(len(data)):
    #strategy.compute_tick(i)
    if data["RSI"][i]<sobreVenta:
        strategy.entry("Compra", i)
    if data["RSI"][i]>=sobreCompra:
        strategy.close_all(i)
  
  strategy.end(i)
  #strategy.print_report()
  ganancia = strategy.get_result()
  del strategy

  return ganancia

In [11]:
  #result = calc_strategy_rsi(price_dict[ticker], 23, 85, "Close")
  #print(sv_max, sc_max, result_max)

In [12]:
for ticker in tickers:
  result = 0
  sc_max = 0
  sv_max = 0
  result_max = 0
  print("\nCalculando estrategia para ",ticker)
  for sv in range(5, 45):
    for sc in range(55, 95):
      result = calc_strategy_rsi(price_dict[ticker], sv, sc, "Close")
      if result > result_max:
        result_max = result
        sc_max = sc
        sv_max = sv
  print(sv_max, sc_max, result_max)


Calculando estrategia para  GGAL
23 85 1204693.3097128868

Calculando estrategia para  BMA
26 84 524282.0716381073

Calculando estrategia para  SUPV
17 76 199252.62387561798

Calculando estrategia para  BBAR
32 84 229101.53616285324
